In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow_addons import metrics

In [4]:
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

In [23]:
labeled_dataset=pd.read_csv("FB_posts_labeled.txt",delimiter="\t")

In [24]:
unlabeled_dataset=pd.read_csv("FB_posts_unlabeled.txt",delimiter="\t")

In [25]:
pred_text=np.array(unlabeled_dataset['message'])

In [26]:
text=np.array(labeled_dataset['message'])

In [27]:
label=np.array(labeled_dataset[['Appreciation','Complaint','Feedback']])


In [28]:
label

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [3]:
labeled_dataset.head()

,postId,message,Appreciation,Complaint,Feedback
0,126016648090_10150802142013091,Great ! ;),1,0,0
1,108381603303_10151136215833304,YUM! YUM!,1,0,0
2,108381603303_3913438087739,Yummm :)),1,0,0
3,110455108974424_343049739048292,sweet,1,0,0
4,110455108974424_350358541650745,nice,1,0,0


In [38]:
vectorize_layer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens = None,
    standardize = 'lower_and_strip_punctuation',
    split = 'whitespace',
    ngrams = None,
    output_mode = 'int',
    output_sequence_length = None
)
vectorize_layer.adapt(np.array(labeled_dataset['message']))

In [20]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'i',
 'and',
 'a',
 'you',
 'of',
 'for',
 'my',
 'in',
 'that',
 'is',
 'your',
 'it',
 'on',
 'have',
 'with',
 'was',
 'not',
 'this',
 'me',
 'they',
 'are',
 'at',
 'be',
 'from',
 'will',
 'so',
 'we',
 'get',
 'but',
 'if',
 'do',
 'as',
 'just',
 'all',
 'would',
 'one',
 'no',
 'when',
 'an',
 'can',
 'out',
 'or',
 'had',
 'store',
 'customer',
 'there',
 'what',
 'our',
 'up',
 'dont',
 'service',
 'am',
 'them',
 'like',
 'now',
 'bank',
 'their',
 'about',
 'time',
 'been',
 'by',
 'has',
 'how',
 'know',
 'its',
 'us',
 'why',
 'flight',
 'card',
 'buy',
 'because',
 'back',
 'please',
 'were',
 'love',
 'im',
 'people',
 'who',
 'more',
 'he',
 'only',
 'thank',
 'any',
 'go',
 'she',
 'want',
 'then',
 'best',
 'make',
 'told',
 'new',
 'never',
 'very',
 'money',
 'thanks',
 'over',
 'today',
 'going',
 'after',
 'could',
 'account',
 'customers',
 'got',
 'again',
 'other',
 'help',
 'really',
 'some',
 'business',
 'great',
 'way',
 'he

In [22]:
len(vectorize_layer.get_vocabulary())

19465

In [63]:
model_rnn = keras.Sequential()

model_rnn.add(vectorize_layer)

model_rnn.add(keras.layers.Embedding(
    input_dim = len(vectorize_layer.get_vocabulary()),
    output_dim = 128,
    mask_zero = True
))

model_rnn.add(keras.layers.SimpleRNN(128)) 

model_rnn.add(keras.layers.Dense(3, activation = 'softmax'))

# configure training / optimization
model_rnn.compile(loss = keras.losses.CategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=['accuracy'])


In [64]:
# training with 20% validation and 10 epochs.
model_rnn.fit(x = text, y = label, validation_split = 0.2,
              epochs=50, batch_size = 128)

Epoch 1/50
50/50 [==============================] - 20s 374ms/step - loss: 0.9692 - accuracy: 0.5704 - val_loss: 0.8987 - val_accuracy: 0.6930
Epoch 2/50
50/50 [==============================] - 18s 361ms/step - loss: 0.7227 - accuracy: 0.7481 - val_loss: 0.8208 - val_accuracy: 0.7112
Epoch 3/50
50/50 [==============================] - 18s 359ms/step - loss: 0.3768 - accuracy: 0.8965 - val_loss: 0.6703 - val_accuracy: 0.7426
Epoch 4/50
50/50 [==============================] - 18s 365ms/step - loss: 0.1081 - accuracy: 0.9812 - val_loss: 0.7902 - val_accuracy: 0.6962
Epoch 5/50
50/50 [==============================] - 17s 344ms/step - loss: 0.0413 - accuracy: 0.9950 - val_loss: 0.7561 - val_accuracy: 0.7608
Epoch 6/50
50/50 [==============================] - 17s 343ms/step - loss: 0.0224 - accuracy: 0.9980 - val_loss: 0.8124 - val_accuracy: 0.7514
Epoch 7/50
50/50 [==============================] - 17s 341ms/step - loss: 0.0147 - accuracy: 0.9991 - val_loss: 0.8433 - val_accuracy: 0.7589

In [65]:
model_lstm = keras.Sequential()

model_lstm.add(vectorize_layer)

model_lstm.add(keras.layers.Embedding(
    input_dim = len(vectorize_layer.get_vocabulary()),
    output_dim = 128,
    mask_zero = True
))

model_lstm.add(keras.layers.GRU(128)) 

model_lstm.add(keras.layers.Dense(3, activation = 'softmax'))

# configure training / optimization
model_lstm.compile(loss = keras.losses.CategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=['accuracy'])


In [66]:
# training with 20% validation and 10 epochs.
model_lstm.fit(x = text, y = label, validation_split = 0.2,
              epochs=50, batch_size = 128)

Epoch 1/50
50/50 [==============================] - 37s 671ms/step - loss: 0.9553 - accuracy: 0.5386 - val_loss: 0.6444 - val_accuracy: 0.7382
Epoch 2/50
50/50 [==============================] - 30s 599ms/step - loss: 0.5256 - accuracy: 0.7940 - val_loss: 0.5102 - val_accuracy: 0.8010
Epoch 3/50
50/50 [==============================] - 31s 617ms/step - loss: 0.2517 - accuracy: 0.9160 - val_loss: 0.5984 - val_accuracy: 0.7853
Epoch 4/50
50/50 [==============================] - 31s 631ms/step - loss: 0.1220 - accuracy: 0.9587 - val_loss: 0.7511 - val_accuracy: 0.7897
Epoch 5/50
50/50 [==============================] - 32s 653ms/step - loss: 0.0652 - accuracy: 0.9808 - val_loss: 0.9529 - val_accuracy: 0.7841
Epoch 6/50
50/50 [==============================] - 31s 617ms/step - loss: 0.0383 - accuracy: 0.9887 - val_loss: 1.0995 - val_accuracy: 0.7784
Epoch 7/50
50/50 [==============================] - 33s 656ms/step - loss: 0.0288 - accuracy: 0.9917 - val_loss: 1.1165 - val_accuracy: 0.7627

In [48]:
 def get_prediction_file(model):
    pred=model.predict(pred_text)
    pred=pred.round()
    dataset=unlabeled_dataset.drop('message',axis=1)
    dataset['Appreciation_pred']=pred[:,0]
    dataset['Complaint_pred']=pred[:,1]
    dataset['Feedback_pred']=pred[:,1]
    return dataset

In [126]:
prediction=get_prediction_file(model_lstm)

64/64 [==============================] - 2s 35ms/step


In [127]:
prediction2=get_prediction_file(model_rnn)

64/64 [==============================] - 1s 15ms/step


In [129]:
# prediction2.to_csv("predictions_rnn.csv",index=False)

In [65]:
bert_model_name = 'bert_en_cased_L-12_H-768_A-12' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3


In [66]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [67]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation="softmax", name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [68]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.65379035 0.5198299  0.57074624]], shape=(1, 3), dtype=float32)


In [76]:
loss = tf.keras.losses.CategoricalCrossentropy()

In [79]:
epochs = 10
steps_per_epoch = len(text)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics.F1Score(3))
history = classifier_model.fit(x = text, y = label, validation_split = 0.2,
                               epochs=epochs)

Epoch 1/10
  7/199 [>.............................] - ETA: 1:11:34 - loss: 1.7441 - f1_score: 0.1634

KeyboardInterrupt: 

In [49]:
prediction_BERT=get_prediction_file(classifier_model)

64/64 [==============================] - 50s 781ms/step


In [51]:
prediction_BERT.to_csv("submission_BERT.csv",index=False)